In [330]:
import os

import matplotlib

import numpy as np
import pandas as pd

from scipy import stats

from matplotlib import pyplot as plt

In [331]:
# Rutas de archivos.
path_i = "../data/Indices/"
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)

dates = pd.read_csv(path_d + "onset_CHIRPS.csv", index_col = "Año")
# Se convierten las fechas a días julianos.
for i in [0, 1]:
    dates.iloc[:, i] = dates.iloc[:, i].apply(
        lambda x: pd.to_datetime(x) ).apply(lambda x: x.dayofyear)
dates.iloc[:, 2] = dates.iloc[:, 2].apply(lambda x: x[:-5]).astype(int)

cols = ["Año"] + [i for i in range(1, 13)]

BEST_1 = pd.read_table( path_i + "BEST_1M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]
BEST_3 = pd.read_table( path_i + "BEST_3M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]
BEST_5 = pd.read_table( path_i + "BEST_5M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]

BEST_1_Y = BEST_1.mean(axis = 1)
BEST_3_Y = BEST_3.mean(axis = 1)
BEST_5_Y = BEST_5.mean(axis = 1)

BEST = [BEST_1, BEST_3, BEST_5]
BEST_Y = [BEST_1_Y, BEST_3_Y, BEST_5_Y]

# Periodo de media móvil a usar.
m = 0

# Gaussian hypothesis test

In [332]:
# Onset

c = 0

# Confianza
a = 0.05

for i in range(c, c + 1):

    print("\n" + dates.columns[i])

    print("\nYearly correlations:")
    for k in range(0, 3):
        corr = stats.pearsonr( BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        if corr[1] < a: print( f"Year {-k:-2}: " + f"{corr[0]:-8.5f}" )

    for k in range(0, 3):
        print(f"\nMonthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr = stats.pearsonr( BEST[m].shift(k).iloc[k:, j],
                dates.iloc[k:, i] )
            if corr[1] < a: print( f"{ j + 1 :02d}: " + f"{corr[0]:-8.5f}" )


Cuenca - Onset

Yearly correlations:
Year  0: -0.32967

Monthly correlations, year  0:
07: -0.35462
09: -0.36427
10: -0.38581
11: -0.32127

Monthly correlations, year -1:

Monthly correlations, year -2:


In [333]:
# Withdrawal.

c = 1

# Confianza
a = 0.05

for i in range(c, c + 1):

    print("\n" + dates.columns[i])

    print("\nYearly correlations:")
    for k in range(0, 3):
        corr = stats.pearsonr( BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        if corr[1] < a: print( f"Year {-k:-2}: " + f"{corr[0]:-8.5f}" )

    for k in range(0, 3):
        print(f"\nMonthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr = stats.pearsonr( BEST[m].shift(k).iloc[k:, j],
                dates.iloc[k:, i] )
            if corr[1] < a: print( f"{ j + 1 :02d}: " + f"{corr[0]:-8.5f}" )


Cuenca - Withdrawal

Yearly correlations:
Year  0:  0.37726

Monthly correlations, year  0:
08:  0.33095
09:  0.41334
10:  0.46044
11:  0.48023
12:  0.46810

Monthly correlations, year -1:
03: -0.38750

Monthly correlations, year -2:
10: -0.36445
11: -0.39248
12: -0.32637


In [334]:
# Length.

c = 2

# Confianza
a = 0.05

for i in range(c, c + 1):

    print("\n" + dates.columns[i])

    print("\nYearly correlations:")
    for k in range(0, 3):
        corr = stats.pearsonr( BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        if corr[1] < a: print( f"Year {-k:-2}: " + f"{corr[0]:-8.5f}" )

    for k in range(0, 3):
        print(f"\nMonthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr = stats.pearsonr( BEST[m].shift(k).iloc[k:, j],
                dates.iloc[k:, i] )
            if corr[1] < a: print( f"{ j + 1 :02d}: " + f"{corr[0]:-8.5f}" )


Cuenca - Length

Yearly correlations:
Year  0:  0.47230

Monthly correlations, year  0:
05:  0.34392
06:  0.31943
07:  0.42337
08:  0.41248
09:  0.51939
10:  0.56604
11:  0.53946
12:  0.52126

Monthly correlations, year -1:
01: -0.32571
03: -0.33925

Monthly correlations, year -2:
10: -0.34721
11: -0.36818


# Bootstrapping

In [335]:
# Correlation boostrapping.

def pcorr(x, y, axis):
    if x.ndim == 1: 
        return stats.pearsonr(x, y)[0]
    else:
        n = y.shape[0]
        statistic = np.empty( (n,) )
        for i in range(n): statistic[i], _ = stats.pearsonr(x[i, :], y[i, :])
        return statistic

In [336]:
# Onset.

c = 0

# Confianza
a = 0.05

for i in range(c, c + 1):

    print(dates.columns[i])

    print("Yearly correlations:")
    for k in range(0, 3):
        corr, _ = stats.pearsonr( 
            BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        int = stats.bootstrap( 
            [ BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] ],
            pcorr, confidence_level = 0.95, method = "basic" 
            ).confidence_interval
        if ( corr >= int[0] ) and ( corr <= int[1] ):
            print( f"Year {-k:-2}: " + f"{corr:-8.5f}" )

    for k in range(0, 2):
        print(f"Monthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr, _ = stats.pearsonr(
                BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] )
            int = stats.bootstrap( 
                [ BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] ],
                pcorr, confidence_level = 0.95, method = "basic" 
                ).confidence_interval
            if ( corr >= int[0] ) and ( corr <= int[1] ):
                print( f"{ j + 1 :02d}: " + f"{corr:-8.5f}" )

Cuenca - Onset
Yearly correlations:
Year -1:  0.04766
Year -2:  0.02093
Monthly correlations, year  0:
01: -0.14905
02:  0.02185
03: -0.11716
04: -0.17284
05: -0.23353
06: -0.22384
08: -0.28627
12: -0.30588
Monthly correlations, year -1:
01:  0.16255
02:  0.12779
03:  0.10239
04:  0.21924
05:  0.11398
06:  0.06113
07: -0.03643
08: -0.03814
09: -0.14288
10: -0.05983
11: -0.05714
12: -0.03888


In [337]:
# Withdrawal.

c = 1

# Confianza
a = 0.05

for i in range(c, c + 1):

    print(dates.columns[i])

    print("Yearly correlations:")
    for k in range(0, 3):
        corr, _ = stats.pearsonr( 
            BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        int = stats.bootstrap( 
            [ BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] ],
            pcorr, confidence_level = 0.95, method = "basic" 
            ).confidence_interval
        if ( corr >= int[0] ) and ( corr <= int[1] ):
            print( f"Year {-k:-2}: " + f"{corr:-8.5f}" )

    for k in range(0, 2):
        print(f"Monthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr, _ = stats.pearsonr(
                BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] )
            int = stats.bootstrap( 
                [ BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] ],
                pcorr, confidence_level = 0.95, method = "basic" 
                ).confidence_interval
            if ( corr >= int[0] ) and ( corr <= int[1] ):
                print( f"{ j + 1 :02d}: " + f"{corr:-8.5f}" )

Cuenca - Withdrawal
Yearly correlations:
Year -1: -0.16474
Year -2: -0.22087
Monthly correlations, year  0:
01: -0.03137
02:  0.08447
03:  0.07518
04:  0.17685
05:  0.28054
06:  0.25437
07:  0.28546
Monthly correlations, year -1:
02: -0.24789
04: -0.16814
05: -0.07167
06: -0.09258
07: -0.02430
08:  0.01424
09:  0.00154
10: -0.04777
11: -0.06854
12:  0.06258


In [338]:
# Length.

c = 2

# Confianza
a = 0.05

for i in range(c, c + 1):

    print(dates.columns[i])

    print("Yearly correlations:")
    for k in range(0, 3):
        corr, _ = stats.pearsonr( 
            BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        int = stats.bootstrap( 
            [ BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] ],
            pcorr, confidence_level = 0.95, method = "basic" 
            ).confidence_interval
        if ( corr >= int[0] ) and ( corr <= int[1] ):
            print( f"Year {-k:-2}: " + f"{corr:-8.5f}" )

    for k in range(0, 2):
        print(f"Monthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr, _ = stats.pearsonr(
                BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] )
            int = stats.bootstrap( 
                [ BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] ],
                pcorr, confidence_level = 0.95, method = "basic" 
                ).confidence_interval
            if ( corr >= int[0] ) and ( corr <= int[1] ):
                print( f"{ j + 1 :02d}: " + f"{corr:-8.5f}" )

Cuenca - Length
Yearly correlations:
Year -1: -0.14688
Year -2: -0.16917
Monthly correlations, year  0:
01:  0.07148
02:  0.04571
03:  0.12642
04:  0.23289
Monthly correlations, year -1:
02: -0.25698
04: -0.25933
05: -0.12370
06: -0.10458
07:  0.00621
08:  0.03451
09:  0.09273
10:  0.00464
11: -0.01174
12:  0.06912
